In [1]:
import numpy as np
from pygsti.baseobjs import Basis

np.set_printoptions(precision=1, linewidth=1000)

In [2]:
import pygsti
from pygsti.extras import idletomography as idt

n_qubits = 1
gates = ["Gi","Gx","Gy","Gcnot"]
max_lengths = [1,2,4,8]
pspec = pygsti.processors.QubitProcessorSpec(n_qubits, gates, geometry='line', nonstd_gate_unitaries={():1})

mdl_target = pygsti.models.create_crosstalk_free_model(pspec)
paulidicts = idt.determine_paulidicts(mdl_target)
idle_experiments = idt.make_idle_tomography_list(n_qubits, max_lengths, paulidicts, maxweight=1)
print(len(idle_experiments), "idle tomography experiments for %d qubits" % n_qubits)

36 idle tomography experiments for 1 qubits


In [3]:
from pygsti.baseobjs import Label
updated_ckt_list= []
for ckt in idle_experiments:
    new_ckt= ckt.copy(editable=True)
    for i,lbl in enumerate(ckt):
        if lbl == Label(()):
            new_ckt[i] = Label(('Gi',0))
    updated_ckt_list.append(new_ckt)

In [4]:
mdl_datagen = pygsti.models.create_crosstalk_free_model(pspec,
                                                        lindblad_error_coeffs={'Gi': {'HX': 0.01}})
# Error models! Random with right CP constraints from Taxonomy paper
ds = pygsti.data.simulate_data(mdl_datagen, updated_ckt_list,100000, seed=8675309)

C:\Users\jkskolf\pyGSTi\pygsti\circuits\circuit.py:816: UserWarning: Editable circuit is being converted to read-only mode in order to hash it.  You should call circuit.done_editing() beforehand.
  _warnings.warn(("Editable circuit is being converted to read-only"


In [5]:
#Run idle tomography (takes ~1.5min w/10Q)
results = idt.do_idle_tomography(n_qubits, ds, max_lengths, paulidicts, maxweight=1, advanced_options={"jacobian mode":"together"}, idle_string='Gi:0')
idt.create_idletomography_report(results,"../IDTTestReport","Test idle tomography example report", auto_open=True)

*** Creating workspace ***
*** Generating switchboard ***
*** Generating tables ***
*** Generating plots ***
*** Merging into template file ***
*** Report Generation Complete!  Total time 0.769244s ***


In [6]:
results.error_list

[NQPauliOp[ X], NQPauliOp[ Y], NQPauliOp[ Z]]

In [7]:
ws = pygsti.report.Workspace()
#ws.init_notebook_mode(autodisplay=True)

In [8]:
print(results)

Idle Tomography Results
Intrinsic stochastic rates: 
   X: 0.000708459
   Y: -3.04705e-05
   Z: 3.04705e-05
Intrinsic affine rates: 
   X: -3.48486e-21
   Y: -1.37661e-05
   Z: 9.50637e-06
Intrinsic hamiltonian rates:
   X: 0.0203293
   Y: -0.000755944
   Z: 0.00027426



In [9]:
print(ds)

Gy:0Gi:0Gy:0@(0)  :  {('0',): 0.0, ('1',): 100000.0}
Gy:0Gi:0Gi:0Gy:0@(0)  :  {('0',): 0.0, ('1',): 100000.0}
Gy:0Gi:0Gi:0Gi:0Gi:0Gy:0@(0)  :  {('0',): 0.0, ('1',): 100000.0}
Gy:0Gi:0Gi:0Gi:0Gi:0Gi:0Gi:0Gi:0Gi:0Gy:0@(0)  :  {('0',): 0.0, ('1',): 100000.0}
Gx:0Gi:0Gx:0@(0)  :  {('0',): 9.0, ('1',): 99991.0}
Gx:0Gi:0Gi:0Gx:0@(0)  :  {('0',): 39.0, ('1',): 99961.0}
Gx:0Gi:0Gi:0Gi:0Gi:0Gx:0@(0)  :  {('0',): 175.0, ('1',): 99825.0}
Gx:0Gi:0Gi:0Gi:0Gi:0Gi:0Gi:0Gi:0Gi:0Gx:0@(0)  :  {('0',): 655.0, ('1',): 99345.0}
Gi:0@(0)  :  {('0',): 99985.0, ('1',): 15.0}
Gi:0Gi:0@(0)  :  {('0',): 99958.0, ('1',): 42.0}
Gi:0Gi:0Gi:0Gi:0@(0)  :  {('0',): 99847.0, ('1',): 153.0}
Gi:0Gi:0Gi:0Gi:0Gi:0Gi:0Gi:0Gi:0@(0)  :  {('0',): 99386.0, ('1',): 614.0}
Gy:0Gy:0Gy:0Gi:0Gy:0Gy:0Gy:0@(0)  :  {('0',): 0.0, ('1',): 100000.0}
Gy:0Gy:0Gy:0Gi:0Gi:0Gy:0Gy:0Gy:0@(0)  :  {('0',): 0.0, ('1',): 100000.0}
Gy:0Gy:0Gy:0Gi:0Gi:0Gi:0Gi:0Gy:0Gy:0Gy:0@(0)  :  {('0',): 0.0, ('1',): 100000.0}
Gy:0Gy:0Gy:0Gi:0Gi:0Gi:0Gi:0Gi:0Gi:0Gi